In [ ]:
# pip install python-dateutil requests

import requests
import base64
import json
import tempfile
import zipfile
from pathlib import Path
from datetime import datetime  
from datetime import timedelta 

In [ ]:
def download_file(url, local_filename, custonHeader, checkCert=True):
    print(f'{url} started')
    with requests.get(url,headers=custonHeader, stream=True, verify=checkCert ) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):  
                f.write(chunk)

    print(f'download {url} done')
    return local_filename

![sc1](http:///www.plantuml.com/plantuml/png/TO_DQiCm48Jl-nHBxhdq6708j5Mf-qF9Ztifb6syf097KgML8NtwUXE5EfJwOCJEDyFC8Wb5oViEQ2i-ukdR93u_AU46mi3OhGta1EiSh71yf0_LQhWGc-57Eox6vaARcDLqVVioAQ81sE1qhWbLXSP7JdBjUuvOLL1FvyqCxtX8j2rxV6AXbeIWA2jjFJPfVTkJC3O0dVT1pOf9Jj0SFSliz7WoGKaQO-nQQ7aKD4AMd78dwNYgPvZTjmqOor47hAxUds-QUoslZ1awCiNj9tvbZWVCoRfT6METSEofGtBjctjzM6FmtM7dNN6MTIKHVgUPFykCyhhiJ9U4_rSPACXurMA0yhxx0G00)

In [ ]:
INPUTs={}
INPUTs['NumFrames']=1
INPUTs['identifier']='VX0200013a'
INPUTs['ForestCoverServiceId']='https://vantage.earthi.world/secure/api/v2.0/services/20'
INPUTs['wkt']=None

# Vantage

## Set Vantage User configurations

In [ ]:
USERNAME=input("Insert Vantage username: ")
TOKEN=input("Insert Vantage token:")
basic=USERNAME+':'+TOKEN
AUTHORIZATION="Basic " + base64.b64encode(bytes(basic, 'utf-8')).decode('utf-8')
HEADERS={'Authorization':AUTHORIZATION,'Content-Type':'application/hal+json;charset=UTF-8'}

# Retrieve the video info

We need:
1. File **URI** for Vantage 
1. Video **startDate** for FoodSecurity Tep
1. Video **completionDate**  for FoodSecurity Tep

## Get video metadata /secure/api/v2.0/search

We use the video **identifier(VX0200013a)** to retrieve the metadata

In [ ]:
url_search_metadata='https://vantage.earthi.world/secure/api/v2.0/search?catalogue=REMOTE_DATA_commercialData&geometry=&identifier='+INPUTs['identifier']+'&remoteDataCollection_commercialData=eovep2e8134bcdaf74f528b23b6dba883bcde'
search_result = requests.get(url_search_metadata,headers=HEADERS)

In [ ]:
search_result_dict = json.loads(search_result.text)
found=False
for i in search_result_dict['features']:
    productIdentifier = i['properties']['productIdentifier']
    if productIdentifier == INPUTs['identifier']:
        INPUTs['startDate']=i['properties']['startDate'].split('.')[0]
        INPUTs['completionDate']=i['properties']['completionDate'].split('.')[0]
        INPUTs['VideoFile']=i['properties']['platformUrl']
        found=True
        break
    
if found is False:
    print("Video identifier NOT FOUND")
    quit()
else:
    print(f'Start Date: {INPUTs["startDate"]}')
    print(f'CompletionDate: {INPUTs["completionDate"]}')
    print(f'Video Uri: {INPUTs["VideoFile"]}')

**Adds 7 days before and after the video date properties. These dates will be use in FS-TEP**

In [ ]:
INPUTs['startDate']=datetime.fromisoformat(INPUTs["startDate"])-timedelta(days=7)
INPUTs['completionDate']=datetime.fromisoformat(INPUTs["completionDate"])+timedelta(days=7)
INPUTs

# Start the service

1. **Creating the configuration**;
1. **Start**;

## Creating the configuration(POST /secure/api/v2.0/jobConfigs)

In [ ]:
payload=f'{{"service": "{INPUTs["ForestCoverServiceId"]}", "inputs":{{"VideoFile":[ "{INPUTs["VideoFile"]}" ],"NumFrames":[{INPUTs["NumFrames"]}]}}, "parent":null}}'
payload

In [ ]:
url='https://vantage.earthi.world/secure/api/v2.0/jobConfigs'
r = requests.post(url,data=str(payload),headers=HEADERS)

In [ ]:
if r.status_code != 201:
    print("Request preparation failed!!")
    quit()

In [ ]:
r.json()

In [ ]:
resp_dict = json.loads(r.text)
IDCONFIG=resp_dict["id"]
print(f'Responce ID: {IDCONFIG}')

## Start(/secure/api/v2.0/jobConfigs/{IDCONFIG}/launch)

In [ ]:
url=f'https://vantage.earthi.world/secure/api/v2.0/jobConfigs/{IDCONFIG}/launch'
run_request = requests.post(url,headers=HEADERS)

**Checks return code**

In [ ]:
run_request_dict = json.loads(run_request.text)
JOB_REQUEST=run_request.status_code
JOB_STATUS=run_request_dict["content"]["status"]
JOB_ID=run_request_dict["content"]["id"]

if JOB_REQUEST != 202:
    print("Run error!")
    quit()

print(f'Request Status code: {JOB_REQUEST} Job status: {JOB_STATUS} JobId: {JOB_ID}')

## GET STATUS(/secure/api/v2.0/jobs/{JOB_ID})

In [ ]:
url_status=f'https://vantage.earthi.world/secure/api/v2.0/jobs/{JOB_ID}'
status = requests.get(url_status,headers=HEADERS)

url_status_dict=json.loads(status.text)
STATUS_REQUEST=status.status_code

if STATUS_REQUEST != 200:
    print("Request status error!")
    quit()

JOB_STATUS=url_status_dict["status"]
JOB_PHASE=url_status_dict["phase"]
JOB_STAGE=url_status_dict["stage"]

print(f'Request Status code: {STATUS_REQUEST} Job status: {JOB_STATUS} Job phase: {JOB_PHASE} Job stage: {JOB_STAGE}')

**Repeat the last step until the service is complete or fails**

# GET RESULTS

In [ ]:
output_dict = json.loads(status.text)
url=str(output_dict['_links']['outputFiles']['href'])
print(f'Get download files info: {url}')

In [ ]:
output_files = requests.get(url,headers=HEADERS)
resp_dict = json.loads(output_files.text)
resp_dict

In [ ]:
ZIP_FILE=None
for down in resp_dict['_embedded']['platformFiles']:
    local_filename = down['filename'].split('/')[-1]
    file_to_download = "https://vantage.earthi.world/secure/api/v2.0/platformFiles/" + str(down['id'])+"/dl"
    print(f'File path: {file_to_download} filename: {local_filename}')
    download_file(file_to_download,local_filename,HEADERS)
    ZIP_FILE=local_filename
    
print('Files downloaded')

## Unzip result, Retrieve WKT info

In [ ]:
print(f'Unzip result file {ZIP_FILE}')
tmpf = tempfile.TemporaryDirectory(suffix='_unzip', prefix='unzip_')

print(f'extract wkt from {ZIP_FILE}' )
with zipfile.ZipFile(ZIP_FILE, 'r') as zipObject:
    listOfFileNames = zipObject.namelist()
    for fileName in listOfFileNames:
        if fileName.endswith('.wkt'):
            print(f'extract file: {fileName}')
            zipObject.extract(fileName, tmpf.name)
            INPUTs['wkt'] = Path(f'{tmpf.name}/{fileName}').read_text().rstrip()
            print(f'>{INPUTs["wkt"]}<')        
tmpf.cleanup()   
if INPUTs['wkt'] is None:
    print("wkt not found ")
    quit()

## Inputs updated

In [ ]:
INPUTs

# FS-TEP

## Set FSTEP USER configurations

In [ ]:
USERNAME_FSTEP=input("Insert FS-TEP username: ")
TOKEN_FSTEP=input("Insert FS-TEP token:")
basic=f'{USERNAME_FSTEP}:{TOKEN_FSTEP}'
AUTHORIZATION="Basic " + base64.b64encode(bytes(basic, 'utf-8')).decode('utf-8')
HEADERS={'Authorization':AUTHORIZATION,'Content-Type':'application/hal+json;charset=UTF-8'}
HEADERS

## S2_NDVI parameter

The NDVI service requires a **Sentinel-2 tile**, we can retrieve tile using the FS-TEP search API: */secure/api/v2.0/search*

1. WKT polygon
1. Video Start date
1. Video Completion date

In [ ]:
PROCESSING_LEVEL='L1C'
MISSION='sentinel2'
CATALOG_TYPE='SATELLITE'
MAX_CLOUD_COVER='10'
PRODUCT_END=str(INPUTs['completionDate'].strftime('%Y-%m-%dT%H:%M:%SZ'))
PRODUCT_START=str(INPUTs['startDate'].strftime('%Y-%m-%dT%H:%M:%SZ'))
WKT=INPUTs["wkt"]

url=f'https://foodsecurity-tep.net/secure/api/v2.0/search?aoi={WKT}&catalogue={CATALOG_TYPE}&maxCloudCover={MAX_CLOUD_COVER}&mission={MISSION}&productDateEnd={PRODUCT_END}&productDateStart={PRODUCT_START}&s2ProcessingLevel={PROCESSING_LEVEL}'
url

**Get results info**

In [ ]:
search_request = requests.get(url,headers=HEADERS, verify=False)

In [ ]:
results_dict = json.loads(search_request.text)
TOTAL_RESULT = results_dict['page']['totalElements']
print(f'Results: {TOTAL_RESULT}')

**Get product identifier from results**

In [ ]:
INPUTs['sentinel2Input']=None
for i in results_dict['features']:
    print(f'ProductIdentifier: {i["properties"]["productIdentifier"]} platformUrl: {i["properties"]["platformUrl"]}')
    if INPUTs['sentinel2Input'] is None:
          INPUTs['sentinel2Input']=i["properties"]["platformUrl"]        

In [ ]:
INPUTs['sentinel2Input']='sentinel2:///S2A_MSIL1C_20220225T111051_N0400_R137_T31VEH_20220225T113038.SAFE'

## RUN S2_NDVI on FSTEP

## Creating the configuration...

In [ ]:
payload=f'{{"service":"https://foodsecurity-tep.net/secure/api/v2.0/services/20","inputs":{{"parallelInputs":["{INPUTs["sentinel2Input"]}"],"collection":[""]}},"parent":null}}'
url='https://foodsecurity-tep.net/secure/api/v2.0/jobConfigs'
request_config_fstep = requests.post(url,data=str(payload),headers=HEADERS, verify=False)

**Remember to check the http_status_cod**

In [ ]:
request_config_fstep_dict=json.loads(request_config_fstep.text)
IDCONFIG=request_config_fstep_dict['id']
print(f'Config id: {IDCONFIG}')

## Start...

In [ ]:
url=f'https://foodsecurity-tep.net/secure/api/v2.0/jobConfigs/{IDCONFIG}/launch'
run_request_fstep = requests.post(url,headers=HEADERS, verify=False)

**Remember to check the http_status_code**

In [ ]:
run_dict=json.loads(run_request_fstep.text)

JOB_STATUS=run_dict["content"]["status"]
JOB_ID=run_dict["content"]["id"]

print(f'Job status: {JOB_STATUS} JobId: {JOB_ID}')

## Get status(/secure/api/v2.0/jobs/{JOB_ID})

In [ ]:
url_status=f'https://foodsecurity-tep.net/secure/api/v2.0/jobs/{JOB_ID}'
status = requests.get(url_status,headers=HEADERS,verify=False)

url_status_dict=json.loads(status.text)
STATUS_REQUEST=status.status_code

JOB_STATUS=url_status_dict["status"]
JOB_PHASE=url_status_dict["phase"]
JOB_STAGE=url_status_dict["stage"]
JOB_ID_CONFIRM=url_status_dict["id"]

status.text

#print(f'Request Status code: {STATUS_REQUEST} JobId: {JOB_ID_CONFIRM} Job status: {JOB_STATUS} Job phase: {JOB_PHASE} Job stage: {JOB_STAGE}')

**Repeat the last tep until the service is complete or fails**

## Get final result

In [ ]:
output_dict = json.loads(status.text)
url=str(output_dict['_links']['outputFiles']['href'])
print(f'Get download files info: {url}')
#request
output_files = requests.get(url,headers=HEADERS,verify=False)
resp_dict = json.loads(output_files.text)

for down in resp_dict['_embedded']['platformFiles']:
    local_filename = down['filename'].split('/')[-1]
    file_to_download = "https://foodsecurity-tep.net/secure/api/v2.0/platformFiles/" + str(down['id'])+"/dl"
    print(f'File path: {file_to_download} filename: {local_filename}')
    download_file(file_to_download,local_filename,HEADERS,False)